In [1]:
import sys
import pandas as pd
from gurobipy import *
import time

In [2]:
start_code = time.time()

In [3]:
data = pd.read_excel('收货.xls')
dist = pd.read_csv('inputdistancetime_5_1201.csv')
node = list(data['ID'])
data.set_index(["ID"], inplace=True)

In [4]:
s = len(data[data['type'] == 4])  #充电桩个数

In [5]:
s

100

In [6]:
#电动车状态
Battery = 120000.0
Capacity = 2.5
Volume = 16
Consume = 1.0
SwapTime = 30.0

Demand = {}
Vol_Demand = {}
ReaTime = {}
DueTime = {}
SerTime = 30.0
Distance = {}
Time = {}

In [7]:
for i in node:
    Demand[i] = data.loc[i,'pack_total_weight']
    Vol_Demand[i] = data.loc[i,'pack_total_volume']
    ReaTime[i] = int(data.loc[i,'first_receive_tm'])
    DueTime[i] = int(data.loc[i,'last_receive_tm'])

In [9]:
ReaTime[50001]

210

In [10]:
#距离和时间
node.append(2000)
for i in dist.index:
    Distance[dist.loc[i,'from_node'],dist.loc[i,'to_node']] = dist.loc[i,'distance']
    Time[dist.loc[i,'from_node'],dist.loc[i,'to_node']] = dist.loc[i,'spend_tm']

In [11]:
Time[0,50001]

68

In [12]:
node[1001]

2000

In [13]:
##回来的点
for i in node:
    if i not in [0,2000]:
        Distance[2000,i] = Distance[0,i]
        Distance[i,2000] = Distance[i,0]
        Time[i,2000] = Time[i,0]
        Time[2000,i] = Time[0,i]
Distance[0,2000] = Distance[2000,0] = 0
Time[0,2000] = Time[2000,0] = 0

In [14]:
Distance[50001,0]

58509

In [15]:
Demand[2000] = Demand[0]
Vol_Demand[2000] = Vol_Demand[0]
ReaTime[2000] = ReaTime[0]
DueTime[2000] = DueTime[0]
n = len(node)
print(node)
print('data ok')

[0, 50001, 50002, 50003, 50004, 50005, 50006, 50007, 50008, 50009, 50010, 50011, 50012, 50013, 50014, 50015, 50016, 50017, 50018, 50019, 50020, 50021, 50022, 50023, 50024, 50025, 50026, 50027, 50028, 50029, 50030, 50031, 50032, 50033, 50034, 50035, 50036, 50037, 50038, 50039, 50040, 50041, 50042, 50043, 50044, 50045, 50046, 50047, 50048, 50049, 50050, 50051, 50052, 50053, 50054, 50055, 50056, 50057, 50058, 50059, 50060, 50061, 50062, 50063, 50064, 50065, 50066, 50067, 50068, 50069, 50070, 50071, 50072, 50073, 50074, 50075, 50076, 50077, 50078, 50079, 50080, 50081, 50082, 50083, 50084, 50085, 50086, 50087, 50088, 50089, 50090, 50091, 50092, 50093, 50094, 50095, 50096, 50097, 50098, 50099, 50100, 50101, 50102, 50103, 50104, 50105, 50106, 50107, 50108, 50109, 50110, 50111, 50112, 50113, 50114, 50115, 50116, 50117, 50118, 50119, 50120, 50121, 50122, 50123, 50124, 50125, 50126, 50127, 50128, 50129, 50130, 50131, 50132, 50133, 50134, 50135, 50136, 50137, 50138, 50139, 50140, 50141, 50142, 50

In [16]:
n

1002

In [17]:
m = Model('VRP')
m.setParam('OutputFlag',True)########
m.setParam('TimeLimit',43200.0)

Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 43200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


In [18]:
###构造变量
#travle or not
x = {}           
for i in node:
    for j in node:
        if i != j:
            x[i,j] = m.addVar(0.0, 1.0, 0.0, GRB.BINARY,'x_%s%s' % (i, j))

In [19]:
#到达时间
ArrTime = {}
for i in node:
    ArrTime[i] = m.addVar(ReaTime[i], DueTime[i], 0.0, GRB.CONTINUOUS, "ArrTime_%s" %i)

In [20]:
#剩余货物
RemainCargo = {}
RemainVelocity = {}
for i in node:
    RemainCargo[i] = m.addVar(0,Capacity,0.0,GRB.CONTINUOUS,'RemainCargo_%s' %i)
for i in node:
    RemainVelocity[i] = m.addVar(0,Volume,0.0,GRB.CONTINUOUS,'RemainVelocity_%s' %i)

In [21]:
#剩余电量
RemainBattery = {}
for i in node:
    RemainBattery[i] = m.addVar(0,Battery, 0.0, GRB.CONTINUOUS,'RemainBattery_%s' %i)
#Integrate new variables
    m.update()

In [22]:
#构造目标函数
obj1 = quicksum(x[0,j] for j in node[1:n]) #车辆数
obj2 = quicksum(Distance[i,j]*x[i,j] for i in node for j in node[1:n] if i != j)  #运输距离
obj3 = quicksum(x[i,j] for i in node[1:s+1] for j in node[s+1:n])      #充电次数

In [23]:
m.setObjective(50*obj3 + 0.014*obj2 + 300*obj1,GRB.MINIMIZE)

In [24]:
##构造约束条件
#1.每个顾客点都被访问
for i in node[s+1:n-1]:
    m.addConstr(quicksum(x[i,j] for j in node[1:n] if i != j) == 1) 

In [25]:
#2.虚拟充电站访问设置
for i in node[1:s+1]:
    m.addConstr(quicksum(x[i,j] for j in node[1:n] if i != j) <= 5) 

In [26]:
#3.从A点进入了就要从A点出去
for h in node[1:n-1]:
    m.addConstr(quicksum(x[h,i] for i in node[1:n] if i != h) - quicksum(x[j,h] for j in node[0:n-1] if h != j) == 0)

In [27]:
#4.时间窗
for j in node[1:n]:
    m.addConstr(ArrTime[0]+(Time[0,j]+0)*x[0,j]-DueTime[0]*(1-x[0,j])-ArrTime[j]<=0)
for i in node[s+1:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(ArrTime[i]+(Time[i,j]+SerTime)*x[i,j]-DueTime[0]*(1-x[i,j])-ArrTime[j]<=0)
for i in node[1:s+1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(ArrTime[i]+(Time[i,j]+SwapTime)*x[i,j]-(DueTime[0]+SwapTime)*(1-x[i,j])-ArrTime[j]<=0)

In [28]:
#5.容量限制
for i in node[0:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainCargo[j]-RemainCargo[i]+Demand[i]*x[i,j]-Capacity*(1-x[i,j])<=0)
for i in node[0:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainVelocity[j]-RemainVelocity[i]+Vol_Demand[i]*x[i,j]-Volume*(1-x[i,j])<=0)

In [29]:
#6.电池容量限制
for i in node[s+1:n-1]:
    for j in node[1:n]:
        if i != j:
            m.addConstr(RemainBattery[j]-RemainBattery[i]+Consume*Distance[i,j]*x[i,j]-Battery*(1-x[i,j])<=0)
for j in node[1:n]:
    m.addConstr(RemainBattery[j]-RemainBattery[0]+Consume*Distance[0,j]*x[0,j]-Battery*(1-x[0,j])<=0)

In [30]:
for i in node[1:s+1]:
    for j in node[1:len(node)]:
        if i != j:
            m.addConstr(RemainBattery[j] +Consume*Distance[i,j]*x[i,j]-Battery<=0)     

In [31]:
#7.其他的一些限制
for i in node[1:n-1]:
    m.addConstr(x[i,0] == 0)
    m.addConstr(x[2000,i] == 0)

In [ ]:
#m.addConstr(x[0,node[1]] == 0)
#m.addConstr(x[node[1],2000] == 0)

In [32]:
m.update()

In [33]:
try:
    m.optimize()
    print(m.ObjVal)
except GurobiError:
    print('Error reported')
print("optimize()")
print(GRB.Status)

Optimize a model with 4008004 rows, 1007010 columns and 14914012 nonzeros
Variable types: 4008 continuous, 1003002 integer (1003002 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e-01, 1e+03]
  Bounds range     [1e+00, 1e+05]
  RHS range        [1e+00, 1e+05]
Presolve removed 56756 rows and 2110 columns (presolve time = 6s) ...
Presolve removed 1767629 rows and 430977 columns (presolve time = 10s) ...
Presolve removed 1767629 rows and 430977 columns (presolve time = 19s) ...
Presolve removed 1767629 rows and 430978 columns (presolve time = 20s) ...
Presolve removed 1767652 rows and 432193 columns (presolve time = 26s) ...
Presolve removed 1767722 rows and 432269 columns (presolve time = 30s) ...
Presolve removed 1767722 rows and 432269 columns (presolve time = 35s) ...
Presolve removed 1767722 rows and 432269 columns (presolve time = 40s) ...
Presolve removed 1767722 rows and 432269 columns (presolve time = 45s) ...
Presolve removed 1767722 rows 

In [34]:
def printTour(solution, route, start, distance, demand,vol_demand,sol_time,solved_point,point):
    route = route + str(start) + ' -> '
    if start not in node[0:s+1]:
        solved_point.append(start)
    point.append(start)
    demand += Demand[start]
    vol_demand +=Vol_Demand[start]
    for i in node:
        if start != i:
            if (solution[start, i]> 0.5) :
                if i not in solved_point and (round(sol_time[i])>=round(sol_time[start]+30+Time[start,i])): 
                    totalDistance = distance + Distance[start,i]
                    if (i == 2000):
                        return [
                        totalDistance,
                        demand,
                        vol_demand,
                        route + '2000',
                        start,        #最后一个点
                        point,
                        solved_point
                        ]
                    return printTour(solution, route, i, totalDistance, demand,vol_demand,sol_time,solved_point,point)         

In [35]:
E = pd.DataFrame()
if True:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    sol_time = m.getAttr("x", ArrTime)

    solved_point = []
    for i in node:
        if i != 0:
            if solution[0,i] >0.5:
                start_time = sol_time[i]-Time[0,i]
                output = printTour(solution,'0 ->',i,Distance[0,i],0,0,sol_time,solved_point,[0])
                solved_point = output[6]                
                points = output[5]
                charge_time = 0
                for p in points:
                    if p in node[1:s+1]:
                        charge_time = charge_time+1
                output = output[0:6] +[start_time,charge_time]  
                if output[0]<=100000 and output[1]<=2 and output[2]<=12:
                    output.append(1)
                else:
                    output.append(2)
                E = E.append([output])

    E.columns= [u'里程',u'总重量',u'总体积',u'顺序',u'最后一点',u'经过的点',u'出发时间',u'充电次数',u'车型']  
    E.to_csv('out.csv',encoding = 'utf-8')

objective: 10000000000000000159028911097599180468360808563945281389781327557747838772170381060813469985856815104.000000


GurobiError: Unable to retrieve attribute 'x'

In [ ]:
end_code = time.time()
print(start_code-end_code)